In [2]:
## Import packages

import pandas as pd

In [3]:
### Import csv files

df_squadValues = pd.read_csv('squad_values.csv')
df_transferDetails = pd.read_csv('transfer_details.csv')
df_standings = pd.read_csv('standings.csv')

In [4]:
### Cleaning the squad values dataframe

market_values = df_squadValues['Market_value'].apply(lambda x: x.replace('€', ''))
market_values = market_values.replace({"th":"*1e3", "m":"*1e6", "bn":"*1e9"}, regex=True).map(pd.eval).astype(int)
df_squadValues["Market_value"] = market_values
df_squadValues = df_squadValues.rename(columns={"Year": "Season"})
df_squadValues['Team'] = df_squadValues["Team"].apply(lambda x: x.strip())


In [5]:
### Cleaning the transfer details dataframe

df_transferDetails['Expenditure'] = df_transferDetails['Expenditure'].apply(lambda x: x.replace('€', ''))
df_transferDetails['Income'] = df_transferDetails['Income'].apply(lambda x: x.replace('€', ''))

df_transferDetails['Expenditure'] = df_transferDetails['Expenditure'].replace("-", 0).replace({"Th.":"*1e3", "m":"*1e6", "bn":"*1e9"}, regex=True).map(pd.eval).astype(int)
df_transferDetails['Income'] = df_transferDetails['Income'].replace("-", 0).replace({"Th.":"*1e3", "m":"*1e6", "bn":"*1e9"}, regex=True).map(pd.eval).astype(int)
df_transferDetails['Balance'] = df_transferDetails['Expenditure'] - df_transferDetails['Income']


In [6]:
### Cleaning the df_standings dataframe

df_standings['GF'] = df_standings['Goals'].apply(lambda x: int(x.split(':')[0]))
df_standings['GA'] = df_standings['Goals'].apply(lambda x: int(x.split(':')[1]))

In [7]:
### Merging the dataframes into one

df = df_standings.merge(df_transferDetails, how='left', on=['Season', 'Team']).merge(df_squadValues, how="left", on=["Season", "Team"])

In [8]:
### We create a copy of the initial dataframe

df_copy = df
df_copy = df_copy.sort_values(by=["Team", "Season"]).reset_index(drop=True)

In [9]:
### We modify the 2019 season that was shortened due to COVID and apply the points per game ratio

df_2019 = df_copy.loc[df_copy["Season"] == 2019]
df_2019["PtsperGame"] = df_2019["Points"]/df_2019["Matches"]
df_2019["Points"] = (df_2019["PtsperGame"] * 38).astype('int')
df_copy.loc[df_copy["Season"] == 2019] = df_2019

/var/folders/02/kvdchhy158bcnynphq3prw8h0000gn/T/ipykernel_43261/3109556431.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019["PtsperGame"] = df_2019["Points"]/df_2019["Matches"]
/var/folders/02/kvdchhy158bcnynphq3prw8h0000gn/T/ipykernel_43261/3109556431.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019["Points"] = (df_2019["PtsperGame"] * 38).astype('int')


In [10]:
### We create a new column with the Points gained the year before
### For the clubs that were relegated in L2 or promoted in L1, we take the number of points gained during the same season

Pts_year_before = [df_copy.loc[0].Points]
for i in range(1, len(df_copy)):
    if df_copy.loc[i].Team == df_copy.loc[i-1].Team:
        Pts_year_before.append(df_copy.loc[i-1].Points)
    else:
        Pts_year_before.append(df_copy.loc[i].Points)

df_copy["Pts_year_before"] = Pts_year_before

In [11]:
### Export the data to csv
df_copy.to_csv('L1_squad_standings_cleaned.csv', index=False)


In [16]:
df_copy[["Points", "Market_value", "Pts_year_before"]]

,Points,Market_value,Pts_year_before
0,41,26550000,41
1,40,34100000,41
2,23,28300000,40
3,34,50900000,34
4,80,242550000,80
...,...,...,...
215,58,297500000,67
216,37,216900000,58
217,43,44500000,43
218,48,49750000,43
